In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ.get('SENDGRID_ALT_KEY')

In [2]:
import sendgrid
import os
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))


In [ ]:
# test SendGrid mailing list operations
import sendgrid
import os
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
params = {'page': 1, 'page_size': 1}
response = sg.client.contactdb.recipients.get(query_params=params)
print(response.status_code)
#print(response.body)
#print(response.headers)


In [ ]:
# Sendgrid retrieve recipients
import json
params = {'page':1, 'page_size':1}
response = sg.client.contactdb.recipients.get(query_params=params)
print(response.status_code)
contacts = json.loads(response.body)
emails = [recipient['email'] for recipient in contacts['recipients']]
emails

In [ ]:
response = sg.client.contactdb.recipients.count.get()
print(response.status_code)
print(response.body)

parse index.xml to get latest feed item

In [ ]:
import feedparser

feed = feedparser.parse('../docs/index.xml')
newsletter = feed.entries[0]
print(newsletter.title)
print(newsletter.description)

In [ ]:
import re
from bs4 import BeautifulSoup
soup = BeautifulSoup(newsletter.description, 'html.parser')
sections = soup.find_all('section')
subject = '"subject": "' + newsletter.title + '"'
introduction = '"introduction":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in reversed(sections[0].find_previous_siblings('p'))]) + ']'
#section = '"sections":' + '[' + ",".join([ '{"section":"' + re.sub(r'"', r'\"', str(s)) + '"}' for s in sections]) + ']'
section = []
for s in sections:
    heading = '"heading": "' + re.sub(r'"', r'\"', str(s.find('h3'))) + '"'
    description = '"description": "' + str(s.find('p')) + '"'
    description = re.sub(r'"', r'\"', description)
    paragraphs = '"paragraphs":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in s.find('p').find_next_siblings('p')]) + ']'
    sec = '{"section": {' + heading + ', ' + description + ', ' + paragraphs + '}}'
    section.append(sec)
h_sections = '"sections": [' + ",".join(section) + ']'
handlebar = '{' + subject + ', ' + introduction + ', ' + h_sections + '}'
print(handlebar)


{"subject": "ADCN: Journaling, Stacks vs. Open, Big Data Bowl", "introduction":[], "sections": [{"section": {"heading": "<h3 class=\"anchored\">Journaling with Wearables</h3>", "description": "<p>“Expert in yourself” is not yet a norm but among high achievers in business and sports it is headed that way. Type A personalities drive the connection between data and habits since the technology is not quite easy enough to use. Call this stuff more of a really-want-to than a need-to at this point but the utility (and probably usability too) increasing dramatically as the benefit to cost ratio increases in the near future. Read on for why the big benefits are coming, and for why personal privacy is a factor.</p>", "paragraphs":[{"p":"<p>Habits are personal behaviors. They defy universal generalization but there is <a href=\"https://knowledge.wharton.upenn.edu/podcast/this-week-in-business/what-machine-learning-reveals-about-forming-a-healthy-habit/\">an emerging consensus</a> that gaining som

In [9]:
import sendgrid
import os
from sendgrid.helpers.mail import *
from dotenv import load_dotenv
load_dotenv()
emails = ["jstenger@uvm.edu"] #..fill as needed
for email in emails:
	sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
	message = Mail(from_email=os.environ.get('SENDGRID_SENDER_EMAIL'), to_emails=email)
	message.dynamic_template_data = eval(handlebar)
	message.template_id = os.environ.get('SENDGRID_TEMPLATE_ID')
	response = sg.send(message)
	print(response.status_code, email)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)